# 对比

In [1]:
import numpy as np
import faiss

In [12]:
d = 64                            
nb = 100000                         
nq = 10000                          
np.random.seed(1234)                
xb = np.random.random((nb, d)).astype('float32')
xb[:, 0] += np.arange(nb) / 1000.   
xq = np.random.random((nq, d)).astype('float32')
xq[:, 0] += np.arange(nq) / 1000.

## IndexFlatL2

In [18]:
index = faiss.IndexFlatL2(d)        

assert index.is_trained
index.add(xb)                       
assert index.is_trained

k = 4                               # k=4 k邻近搜索
D, I = index.search(xq, k)          # 执行搜索
print(I[:5])                        # 最初五次查询的结果
print(I[-5:])                       # 最后五次查询的结果

[[ 381  207  210  477]
 [ 526  911  142   72]
 [ 838  527 1290  425]
 [ 196  184  164  359]
 [ 526  377  120  425]]
[[ 9900 10500  9309  9831]
 [11055 10895 10812 11321]
 [11353 11103 10164  9787]
 [10571 10664 10632  9638]
 [ 9628  9554 10036  9582]]


## IndexIVFFlat

In [14]:
nlist = 100
k = 4
quantizer = faiss.IndexFlatL2(d)  # the other index
index = faiss.IndexIVFFlat(quantizer, d, nlist, faiss.METRIC_L2)
# here we specify METRIC_L2, by default it performs inner-product search

assert not index.is_trained
index.train(xb)
assert index.is_trained

index.add(xb)                  # 添加索引可能会有一点慢
D, I = index.search(xq, k)     # 搜索
print(I[:5])                   # 最初五次查询的结果
print(I[-5:])                  # 最后五次查询的结果

[[ 381  207  210  477]
 [ 526  911  142   72]
 [ 838  527 1290  425]
 [ 196  184  164  359]
 [ 526  377  120  425]]
[[ 9900  9309  9810 10048]
 [11055 10895 10812 11321]
 [11353 10164  9787 10719]
 [10571 10664 10632 10203]
 [ 9628  9554  9582 10304]]


## IndexIVFPQ

In [17]:
nlist = 100
m = 8
k = 4
quantizer = faiss.IndexFlatL2(d)    # 内部的索引方式依然不变
index = faiss.IndexIVFPQ(quantizer, d, nlist, m, 8) # 每个向量都被编码为8个字节大小
                                    
index.train(xb)
index.add(xb)
D, I = index.search(xb[:5], k)      # 测试
print(I)
print(D)

index.nprobe = 10                   # 与以前的方法相比
D, I = index.search(xq, k)          
print(I[:5])                        # 最初五次查询的结果
print(I[-5:])                       # 最后五次查询的结果

[[   0   78  608  159]
 [   1 1063  555  380]
 [   2  304  134   46]
 [   3   64  773  265]
 [   4  288  827  531]]
[[1.6157436 6.1152253 6.4348025 6.564184 ]
 [1.389575  5.6771317 5.9956017 6.486294 ]
 [1.7025063 6.121688  6.189084  6.489888 ]
 [1.8057687 6.5440307 6.6684756 6.859398 ]
 [1.4920276 5.79976   6.190908  6.3791513]]
[[ 399  210  329 1619]
 [1193   39  911  187]
 [1267  197  527  425]
 [ 184  599  466  359]
 [ 828  377  120  416]]
[[ 9900  8746  9853 10437]
 [10494 10507 11373  9014]
 [10719 11291 10424 10138]
 [10122  9638 11113 10630]
 [ 9229 10304  9644 10370]]
